In [1]:
pip install transformers torch


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
torch.cuda.is_available()

False

In [3]:
# model = T5ForConditionalGeneration.from_pretrained("t5-base").to('cuda:0')
# model = T5ForConditionalGeneration.from_pretrained("t5-base")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")


# model = T5ForConditionalGeneration.from_pretrained("t5-small").to('cuda:0')
model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")



In [5]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

path_test= '/content/drive/My Drive/Colab Notebooks/data/test_dataset.pkl'

stories = pickle.load(open(path_test, 'rb'))
print("total number of CNN data: ",len(stories))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
total number of CNN data:  10000


In [10]:
import pandas as pd
df = pd.DataFrame(stories)

def shorten_sen(list_sen, NUM_SEN=2):
  final_sen =[]
  if len(list_sen) < NUM_SEN:
    NUM_SEN = len(list_sen)
  for i in range(NUM_SEN):
    final_sen.append(list_sen[i])
  return '. '.join(final_sen)# + '.'

df['shorten_story'] = df.apply(lambda row: shorten_sen(row['story']), axis=1)
df['shorten_highlights'] = df.apply(lambda row: row['highlights'][0], axis=1)

df['predicted_highlights'] = '-'
df

# df.iloc[0,4]#, df.iloc[0,0] 

,story,highlights,id,shorten_story,shorten_highlights,predicted_highlights
0,[ever hopped in a taxi and sat in traffic for ...,[mellowcab is a threewheeled vehicle that the ...,id_cnn_88698,ever hopped in a taxi and sat in traffic for h...,mellowcab is a threewheeled vehicle that the o...,-
1,[a mother of six has claimed she was threatene...,[amanda moss and husband simon booked family h...,id_dm_26727,a mother of six has claimed she was threatened...,amanda moss and husband simon booked family ho...,-
2,[thomas morris who was strangled by his own ka...,[thomas morris was found dead in home gym in m...,id_dm_205431,thomas morris who was strangled by his own kar...,thomas morris was found dead in home gym in mo...,-
3,[britains princess anne and major winner annik...,"[ra announces first women members, princess an...",id_cnn_47838,britains princess anne and major winner annika...,ra announces first women members,-
4,[it is alleged that michael karkoc was an offi...,[alleged michael karkoc was in the ssled ukran...,id_dm_63684,it is alleged that michael karkoc was an offic...,alleged michael karkoc was in the ssled ukrani...,-
...,...,...,...,...,...,...
9995,[an indonesian lawyer for condemned bali nine ...,[lawyer for bali nine duo makes plea on social...,id_dm_183675,an indonesian lawyer for condemned bali nine d...,lawyer for bali nine duo makes plea on social ...,-
9996,[campaigners stamping antiisrael propalestine ...,[british banknotes marked with message free pa...,id_dm_14557,campaigners stamping antiisrael propalestine m...,british banknotes marked with message free pal...,-
9997,[the solution to ending an abusive relationshi...,[authors say leaving abuse can be a confusing ...,id_cnn_90018,the solution to ending an abusive relationship...,authors say leaving abuse can be a confusing p...,-
9998,[in compensation for a small piece of land tak...,[victorian government pays melbournes scotch c...,id_dm_192809,in compensation for a small piece of land take...,victorian government pays melbournes scotch co...,-


In [11]:
%%time
from tqdm import tqdm


# for article in articles+articles+articles:
for i in tqdm(range(0,10)):
    article= df.loc[i,'shorten_story']
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("summarize: " + article,
                          return_tensors="pt", max_length=200, truncation=True)#.to('cuda:0')
                          # return_tensors="pt", max_length=512, truncation=True).to('cuda:0')

    # generate the summarization output
    outputs = model.generate(
        inputs,
        max_length=30,
        min_length=10,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True)
    df.loc[i,'predicted_highlights']=tokenizer.decode(outputs[0])
    print()

 10%|█         | 1/10 [00:02<00:26,  2.92s/it]

 20%|██        | 2/10 [00:05<00:22,  2.76s/it]

 30%|███       | 3/10 [00:07<00:17,  2.50s/it]

 40%|████      | 4/10 [00:09<00:14,  2.34s/it]

 50%|█████     | 5/10 [00:11<00:11,  2.23s/it]

...

 70%|███████   | 7/10 [00:15<00:06,  2.10s/it]

 80%|████████  | 8/10 [00:17<00:04,  2.12s/it]

 90%|█████████ | 9/10 [00:19<00:02,  2.04s/it]

100%|██████████| 10/10 [00:21<00:00,  2.11s/it]


CPU times: user 20.2 s, sys: 743 ms, total: 20.9 s
Wall time: 21.1 s


In [9]:
loc_save = '/content/drive/My Drive/Colab Notebooks/data'

df.to_csv(loc_save+'/saved_predicted_sum.csv')

In [13]:
index = 2
df.iloc[index,4], df.iloc[index,0],df.iloc[index,1], df.iloc[index,3] 

('thomas morris was found dead in home gym in morriston swansea',
 ['thomas morris who was strangled by his own karate belt in a freak accident while carrying out a martial arts exercise',
  'a teenage fitness fanatic accidentally strangled himself with a karate belt after trying out a martial arts exercise he had seen on the internet an inquest has heard',
  'thomas morris was found dead in his home gym in morriston swansea with the belt around his neck after trying to copy the training exercise',
  'an inquest heard the karate fan had viewed the online training video of martial arts experts using their belt under their chin',
  'the coroner heard that thomas who was also a talented rugby player had shown his sisters the exercise which he hoped would build up his fitness',
  'his mother belinda and his cousins went to look for thomas in the gym in an outbuilding in the garden when she returned from an afternoon out and found him along with the belt wrapped around his neck',
  'detecti

In [81]:
df

,story,highlights,id,shorten_story,shorten_highlights,predicted_highlights
0,[ever hopped in a taxi and sat in traffic for ...,[mellowcab is a threewheeled vehicle that the ...,id_cnn_88698,ever hopped in a taxi and sat in traffic for h...,mellowcab is a threewheeled vehicle that the o...,neil du preez has developed mellowcabs electri...
1,[a mother of six has claimed she was threatene...,[amanda moss and husband simon booked family h...,id_dm_26727,a mother of six has claimed she was threatened...,amanda moss and husband simon booked family ho...,amanda moss of liverpool wanted to take her ch...
2,[thomas morris who was strangled by his own ka...,[thomas morris was found dead in home gym in m...,id_dm_205431,thomas morris who was strangled by his own kar...,thomas morris was found dead in home gym in mo...,thomas morris accidentally strangled himself w...
3,[britains princess anne and major winner annik...,"[ra announces first women members, princess an...",id_cnn_47838,britains princess anne and major winner annika...,ra announces first women members,princess anne and major winner annika sorensta...
4,[it is alleged that michael karkoc was an offi...,[alleged michael karkoc was in the ssled ukran...,id_dm_63684,it is alleged that michael karkoc was an offic...,alleged michael karkoc was in the ssled ukrani...,michael karkoc is accused of being a notorious...
...,...,...,...,...,...,...
9995,[an indonesian lawyer for condemned bali nine ...,[lawyer for bali nine duo makes plea on social...,id_dm_183675,an indonesian lawyer for condemned bali nine d...,lawyer for bali nine duo makes plea on social ...,-
9996,[campaigners stamping antiisrael propalestine ...,[british banknotes marked with message free pa...,id_dm_14557,campaigners stamping antiisrael propalestine m...,british banknotes marked with message free pal...,-
9997,[the solution to ending an abusive relationshi...,[authors say leaving abuse can be a confusing ...,id_cnn_90018,the solution to ending an abusive relationship...,authors say leaving abuse can be a confusing p...,-
9998,[in compensation for a small piece of land tak...,[victorian government pays melbournes scotch c...,id_dm_192809,in compensation for a small piece of land take...,victorian government pays melbournes scotch co...,-


In [ ]:
result="""us president barack obama wants congress to weigh in on whether to use military force in syria.
            obama sent a letter to the heads of the house and senate on saturday night after announcing that 
            he believes military action is the right step to take over the alleged use of chemical weapons"""
len(result)

312

In [ ]:
reference="""syrian official obama climbed to the top of the tree doesnt know how to get down
  obama sends a letter to the heads of the house and senate
  obama to seek congressional approval on military action against syria
  aim is to determine whether cw were used not by whom says un spokesman"""

In [ ]:
# !pip install py-rouge
# !pip install --user -U nltk

In [ ]:
import rouge


def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


for aggregator in ['Avg', 'Best', 'Individual']:
    print('Evaluation with {}'.format(aggregator))
    apply_avg = aggregator == 'Avg'
    apply_best = aggregator == 'Best'

    evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                           max_n=4,
                           limit_length=True,
                           length_limit=100,
                           length_limit_type='words',
                           apply_avg=apply_avg,
                           apply_best=apply_best,
                           alpha=0.5, # Default F1_score
                           weight_factor=1.2,
                           stemming=True)


    # hypothesis_1 = "King Norodom Sihanouk has declined requests to chair a summit of Cambodia 's top political leaders , saying the meeting would not bring any progress in deadlocked negotiations to form a government .\nGovernment and opposition parties have asked King Norodom Sihanouk to host a summit meeting after a series of post-election negotiations between the two opposition groups and Hun Sen 's party to form a new government failed .\nHun Sen 's ruling party narrowly won a majority in elections in July , but the opposition _ claiming widespread intimidation and fraud _ has denied Hun Sen the two-thirds vote in parliament required to approve the next government .\n"
    # references_1 = ["Prospects were dim for resolution of the political crisis in Cambodia in October 1998.\nPrime Minister Hun Sen insisted that talks take place in Cambodia while opposition leaders Ranariddh and Sam Rainsy, fearing arrest at home, wanted them abroad.\nKing Sihanouk declined to chair talks in either place.\nA U.S. House resolution criticized Hun Sen's regime while the opposition tried to cut off his access to loans.\nBut in November the King announced a coalition government with Hun Sen heading the executive and Ranariddh leading the parliament.\nLeft out, Sam Rainsy sought the King's assurance of Hun Sen's promise of safety and freedom for all politicians.",
    #                 "Cambodian prime minister Hun Sen rejects demands of 2 opposition parties for talks in Beijing after failing to win a 2/3 majority in recent elections.\nSihanouk refuses to host talks in Beijing.\nOpposition parties ask the Asian Development Bank to stop loans to Hun Sen's government.\nCCP defends Hun Sen to the US Senate.\nFUNCINPEC refuses to share the presidency.\nHun Sen and Ranariddh eventually form a coalition at summit convened by Sihanouk.\nHun Sen remains prime minister, Ranariddh is president of the national assembly, and a new senate will be formed.\nOpposition leader Rainsy left out.\nHe seeks strong assurance of safety should he return to Cambodia.\n",
    #                 ]
    hypothesis_1 = result
    references_1 = reference

    all_hypothesis = [hypothesis_1]
    all_references = [references_1]

    scores = evaluator.get_scores(all_hypothesis, all_references)

    for metric, results in sorted(scores.items(), key=lambda x: x[0]):
        if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
            for hypothesis_id, results_per_ref in enumerate(results):
                nb_references = len(results_per_ref['p'])
                for reference_id in range(nb_references):
                    print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                    print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
            print()
        else:
            print(prepare_results(results['p'], results['r'], results['f']))
    print()

Evaluation with Avg
	rouge-1:	P: 51.92	R: 51.92	F1: 51.92
	rouge-2:	P: 23.53	R: 23.53	F1: 23.53
	rouge-3:	P: 20.00	R: 20.00	F1: 20.00
	rouge-4:	P: 18.37	R: 18.37	F1: 18.37
	rouge-l:	P: 54.32	R: 54.32	F1: 54.32
	rouge-w:	P: 36.64	R: 21.86	F1: 27.38

Evaluation with Best
	rouge-1:	P: 51.92	R: 51.92	F1: 51.92
	rouge-2:	P: 23.53	R: 23.53	F1: 23.53
	rouge-3:	P: 20.00	R: 20.00	F1: 20.00
	rouge-4:	P: 18.37	R: 18.37	F1: 18.37
	rouge-l:	P: 54.32	R: 54.32	F1: 54.32
	rouge-w:	P: 36.64	R: 21.86	F1: 27.38

Evaluation with Individual
	Hypothesis #0 & Reference #0: 
		rouge-1:	P: 51.92	R: 51.92	F1: 51.92

	Hypothesis #0 & Reference #0: 
		rouge-2:	P: 23.53	R: 23.53	F1: 23.53

	Hypothesis #0 & Reference #0: 
		rouge-3:	P: 20.00	R: 20.00	F1: 20.00

	Hypothesis #0 & Reference #0: 
		rouge-4:	P: 18.37	R: 18.37	F1: 18.37

	Hypothesis #0 & Reference #0: 
		rouge-l:	P: 54.32	R: 54.32	F1: 54.32

	Hypothesis #0 & Reference #0: 
		rouge-w:	P: 36.64	R: 21.86	F1: 27.38




In [ ]:
# import nltk
# nltk.download('punkt')


In [ ]:
%%time
from transformers import pipeline

# using pipeline API for summarization task
summarization = pipeline("summarization",device=0)
original_text = """
Paul Walker is hardly the first actor to die during a production. 
But Walker's death in November 2013 at the age of 40 after a car crash was especially eerie given his rise to fame in the "Fast and Furious" film franchise. 
The release of "Furious 7" on Friday offers the opportunity for fans to remember -- and possibly grieve again -- the man that so many have praised as one of the nicest guys in Hollywood. 
"He was a person of humility, integrity, and compassion," military veteran Kyle Upham said in an email to CNN. 
Walker secretly paid for the engagement ring Upham shopped for with his bride. 
"We didn't know him personally but this was apparent in the short time we spent with him. 
I know that we will never forget him and he will always be someone very special to us," said Upham. 
The actor was on break from filming "Furious 7" at the time of the fiery accident, which also claimed the life of the car's driver, Roger Rodas. 
Producers said early on that they would not kill off Walker's character, Brian O'Connor, a former cop turned road racer. Instead, the script was rewritten and special effects were used to finish scenes, with Walker's brothers, Cody and Caleb, serving as body doubles. 
There are scenes that will resonate with the audience -- including the ending, in which the filmmakers figured out a touching way to pay tribute to Walker while "retiring" his character. At the premiere Wednesday night in Hollywood, Walker's co-star and close friend Vin Diesel gave a tearful speech before the screening, saying "This movie is more than a movie." "You'll feel it when you see it," Diesel said. "There's something emotional that happens to you, where you walk out of this movie and you appreciate everyone you love because you just never know when the last day is you're gonna see them." There have been multiple tributes to Walker leading up to the release. Diesel revealed in an interview with the "Today" show that he had named his newborn daughter after Walker. 
Social media has also been paying homage to the late actor. A week after Walker's death, about 5,000 people attended an outdoor memorial to him in Los Angeles. Most had never met him. Marcus Coleman told CNN he spent almost $1,000 to truck in a banner from Bakersfield for people to sign at the memorial. "It's like losing a friend or a really close family member ... even though he is an actor and we never really met face to face," Coleman said. "Sitting there, bringing his movies into your house or watching on TV, it's like getting to know somebody. It really, really hurts." Walker's younger brother Cody told People magazine that he was initially nervous about how "Furious 7" would turn out, but he is happy with the film. "It's bittersweet, but I think Paul would be proud," he said. CNN's Paul Vercammen contributed to this report.
"""
# summary_text = summarization(original_text)[0]['summary_text']
# print("Summary:", summary_text)

summary_text = summarization("summarization")
print("Summary:", summary_text)


KeyboardInterrupt: ignored

In [ ]:
original_text = """
Paul Walker is hardly the first actor to die during a production. 
But Walker's death in November 2013 at the age of 40 after a car crash was especially eerie given his rise to fame in the "Fast and Furious" film franchise. 
The release of "Furious 7" on Friday offers the opportunity for fans to remember -- and possibly grieve again -- the man that so many have praised as one of the nicest guys in Hollywood. 
"He was a person of humility, integrity, and compassion," military veteran Kyle Upham said in an email to CNN. 
Walker secretly paid for the engagement ring Upham shopped for with his bride. 
"We didn't know him personally but this was apparent in the short time we spent with him. 
I know that we will never forget him and he will always be someone very special to us," said Upham. 
The actor was on break from filming "Furious 7" at the time of the fiery accident, which also claimed the life of the car's driver, Roger Rodas. 
Producers said early on that they would not kill off Walker's character, Brian O'Connor, a former cop turned road racer. Instead, the script was rewritten and special effects were used to finish scenes, with Walker's brothers, Cody and Caleb, serving as body doubles. 
There are scenes that will resonate with the audience -- including the ending, in which the filmmakers figured out a touching way to pay tribute to Walker while "retiring" his character. At the premiere Wednesday night in Hollywood, Walker's co-star and close friend Vin Diesel gave a tearful speech before the screening, saying "This movie is more than a movie." "You'll feel it when you see it," Diesel said. "There's something emotional that happens to you, where you walk out of this movie and you appreciate everyone you love because you just never know when the last day is you're gonna see them." There have been multiple tributes to Walker leading up to the release. Diesel revealed in an interview with the "Today" show that he had named his newborn daughter after Walker. 
Social media has also been paying homage to the late actor. A week after Walker's death, about 5,000 people attended an outdoor memorial to him in Los Angeles. Most had never met him. Marcus Coleman told CNN he spent almost $1,000 to truck in a banner from Bakersfield for people to sign at the memorial. "It's like losing a friend or a really close family member ... even though he is an actor and we never really met face to face," Coleman said. "Sitting there, bringing his movies into your house or watching on TV, it's like getting to know somebody. It really, really hurts." Walker's younger brother Cody told People magazine that he was initially nervous about how "Furious 7" would turn out, but he is happy with the film. "It's bittersweet, but I think Paul would be proud," he said. CNN's Paul Vercammen contributed to this report.
"""

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-xsum")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum")#.to('cuda:0')

In [ ]:
%%time

articles = [article1,article2,article3,article1,article2,article3]
# articles = [original_text]

for article in articles+articles+articles:
# encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("summarize: " + article,
                          return_tensors="pt", max_length=512, truncation=True)#.to('cuda:0')
    # generate the summarization output
    outputs = model.generate(
        inputs,
        # max_length=150,
        max_length=80,
        min_length=40,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True)
    print(tokenizer.decode(outputs[0]))
    print()

Pop star Justin Timberlake and actress Jessica Biel have announced the birth of their first child, a baby boy, who was born on Monday.., according to People magazine, and weighed in at 7lbs 10oz.

Bob Barker, who hosted "The Price Is Right" for 35 years, returned to the game show for the first time in eight years on April Fool's Day, but he didn't miss a beat.

Bomb disposal experts carried out explosions at a cordoned-off garage after suspected explosives were found during anti-terrorism raids in Gloucestershire. The controlled explosions took place at 11.30am on Saturday, after 15 residents spent the night in a nearby hall and 13 vulnerable people were rehoused

Pop star Justin Timberlake and actress Jessica Biel have announced the birth of their first child, a baby boy, who was born on Monday.., according to People magazine, and weighed in at 7lbs 10oz.

Bob Barker, who hosted "The Price Is Right" for 35 years, returned to the game show for the first time in eight years on April Foo

In [ ]:
%%time

from transformers import pipeline

# summarize with BART
summarizer_bart = pipeline(task='summarization', model="facebook/bart-large-cnn", device=0)
# summarize with T5
summarizer_t5 = pipeline(task='summarization', model="t5-base", device=0)

Some weights of T5Model were not initialized from the model checkpoint at t5-base and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



CPU times: user 48.6 s, sys: 8.86 s, total: 57.5 s
Wall time: 2min 13s


In [ ]:
# summarize with T5
summarizer_yuvraj = pipeline(task='summarization', model="yuvraj/summarizer-cnndm", device=0)

In [ ]:
%%time
for article in articles+articles+articles:
  summary_text = summarizer_bart(article, min_length=50, max_length=150, early_stopping=True, length_penalty=2.0, no_repeat_ngram_size=3, num_beams=4)
  print("Summary:", summary_text[0]['summary_text'])

Your max_length is set to 150, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)
Your max_length is set to 150, but you input_length is only 148. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary: Silas was the middle name of Timberlake's maternal grandfather Bill Bomar, who died in 2012. Randall is the musician's own middle name, as well as his father's first. The couple announced the pregnancy in January, with an Instagram post.
Summary: Bob Barker returned to "The Price Is Right" for the first time in eight years. The 91-year-old hosted the show for 35 years before stepping down in 2007. Drew Carey finished up hosting duties on the April 1 edition of the show.


Your max_length is set to 150, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary: Two men arrested on terrorism charges following raids on two properties. 15 residents spent the night in a nearby hall and 13 were rehoused. A 200-metre cordon was placed around the two properties in Cheltenham. A 31-year-old man was also arrested under the Explosives Substances Act.


Your max_length is set to 150, but you input_length is only 148. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


...

Summary: Silas was the middle name of Timberlake's maternal grandfather Bill Bomar, who died in 2012. Randall is the musician's own middle name, as well as his father's first. The couple announced the pregnancy in January, with an Instagram post.
Summary: Bob Barker returned to "The Price Is Right" for the first time in eight years. The 91-year-old hosted the show for 35 years before stepping down in 2007. Drew Carey finished up hosting duties on the April 1 edition of the show.


Your max_length is set to 150, but you input_length is only 106. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary: Two men arrested on terrorism charges following raids on two properties. 15 residents spent the night in a nearby hall and 13 were rehoused. A 200-metre cordon was placed around the two properties in Cheltenham. A 31-year-old man was also arrested under the Explosives Substances Act.


Your max_length is set to 150, but you input_length is only 148. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


Summary: Silas was the middle name of Timberlake's maternal grandfather Bill Bomar, who died in 2012. Randall is the musician's own middle name, as well as his father's first. The couple announced the pregnancy in January, with an Instagram post.
Summary: Bob Barker returned to "The Price Is Right" for the first time in eight years. The 91-year-old hosted the show for 35 years before stepping down in 2007. Drew Carey finished up hosting duties on the April 1 edition of the show.
Summary: Two men arrested on terrorism charges following raids on two properties. 15 residents spent the night in a nearby hall and 13 were rehoused. A 200-metre cordon was placed around the two properties in Cheltenham. A 31-year-old man was also arrested under the Explosives Substances Act.
CPU times: user 19.6 s, sys: 1.45 s, total: 21.1 s
Wall time: 21.1 s


In [ ]:
%%time
for article in articles+articles+articles:
  summary_text = summarizer_t5(article, min_length=50, max_length=100, early_stopping=True, length_penalty=2.0, no_repeat_ngram_size=3, num_beams=4)
  print("Summary:", summary_text[0]['summary_text'])

Summary: Justin Timberlake and Jessica Biel have welcomed their son, Silas Randall Timberlake . it is the first baby for both of them . the couple announced the pregnancy in a statement to People . "Silas was the middle name of Timberlake's maternal grandfather"
Summary: "The Price Is Right" contestants were told to "come on down!" for the first time in eight years . despite being away from the show for most of the past eight years, he didn't seem to miss a beat .
Summary: controlled explosions took place at 11.30am on saturday after 15 residents spent the night in a nearby hall . 13 vulnerable people were rehoused and a 200-metre cordon was placed around the garage . a 52-year-old man was arrested under the Explosive Substances Act . police later found suspected explosives at a house in nearby Up Hatherley .
Summary: Justin Timberlake and Jessica Biel have welcomed their son, Silas Randall Timberlake . it is the first baby for both of them . the couple announced the pregnancy in a sta

In [ ]:
%%time
for article in articles+articles+articles:
  summary_text = summarizer_yuvraj(article, min_length=50, max_length=100, early_stopping=True, length_penalty=2.0, no_repeat_ngram_size=3, num_beams=4)
  print("Summary:", summary_text[0]['summary_text'])

Summary: Justin Timberlake and Jessica Biel announced the arrival of their son, Silas Randall .
Silas was the middle name of Timberlake's maternal grandfather Bill Bomar .
It is the first child for both the pair for both .
They announced the baby in January .
Summary: Bob Barker hosted "The Price Is Right" for 35 years before stepping down in 2007 .
Barker handled the first price-guessing game of the show, the classic "Lucky Seven," before turning over to Carey .
Carey: Barker didn't seem to miss a beat .
Summary: Two men arrested on terrorism charges in Cheltenham .
They were arrested after 15 neighbours spent the night at a hall .
A 200-metre cordon was placed around the garage as bomb disposal teams combed the area .
Police arrested a 31-year-old local man from the Hester's Way area under the Explosive Substances Act .
Summary: Justin Timberlake and Jessica Biel announced the arrival of their son, Silas Randall .
Silas was the middle name of Timberlake's maternal grandfather Bill Bo